In [2]:

import datetime
import numpy as np
import cv2
from itertools import cycle
import pickle
import pathlib
import math
import tqdm
import scipy.io
from matplotlib import pyplot as plt
import scipy.io
import h5py
import re
from lxml import etree as ET
import scipy.signal as sig
import pandas as pd
from scipy.stats import kde
from BlockSync_current import BlockSync
import UtilityFunctions_newOE as uf
from scipy import signal
import bokeh
import seaborn as sns
from matplotlib import rcParams

rcParams['pdf.fonttype'] = 42  # Ensure fonts are embedded and editable
rcParams['ps.fonttype'] = 42  # Ensure compatibility with vector outputs
%matplotlib inline


def bokeh_plotter(data_list, x_axis_list=None, label_list=None, 
                  plot_name='default',
                  x_axis_label='X', y_axis_label='Y',
                  peaks=None, peaks_list=False, export_path=False):
    """Generates an interactive Bokeh plot for the given data vector.
    Args:
        data_list (list or array): The data to be plotted.
        label_list (list of str): The labels of the data vectors
        plot_name (str, optional): The title of the plot. Defaults to 'default'.
        x_axis (str, optional): The label for the x-axis. Defaults to 'X'.
        y_axis (str, optional): The label for the y-axis. Defaults to 'Y'.
        peaks (list or array, optional): Indices of peaks to highlight on the plot. Defaults to None.
        export_path (False or str): when set to str, will output the resulting html fig
    """
    color_cycle = cycle(bokeh.palettes.Category10_10)
    fig = bokeh.plotting.figure(title=f'bokeh explorer: {plot_name}',
                                x_axis_label=x_axis_label,
                                y_axis_label=y_axis_label,
                                plot_width=1500,
                                plot_height=700)

    for i, data_vector in enumerate(data_list):
        
        color = next(color_cycle)
        
        if x_axis_list is None:
            x_axis = range(len(data_vector))
        elif len(x_axis_list) == len(data_list):
            print('x_axis manually set')
            x_axis = x_axis_list[i]
        else:
            raise Exception('problem with x_axis_list input - should be either None, or a list with the same length as data_list')
        if label_list is None:
            fig.line(x_axis, data_vector, line_color=color, legend_label=f"Line {i+1}")
        elif len(label_list) == len(data_list):
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"{label_list[i]}")
        if peaks is not None and peaks_list is True:
            fig.circle(peaks[i], data_vector[peaks[i]], size=10, color=color)

    if peaks is not None and peaks_list is False:
        fig.circle(peaks, data_vector[peaks], size=10, color='red')

    if export_path is not False:
        print(f'exporting to {export_path}')
        bokeh.io.output.output_file(filename=str(export_path / f'{plot_name}.html'), title=f'{plot_name}')
    bokeh.plotting.show(fig)
    

def load_eye_data_2d_w_rotation_matrix(block):
    """
    This function checks if the eye dataframes and rotation dict object exist, then imports them
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    try:
        block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data.csv', index_col=0, engine='python')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data.csv', index_col=0, engine='python')
    except FileNotFoundError:
        print('eye_data files not found, run the pipeline!')
        return
    
    try:
        with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'rb') as file:
            rotation_dict = pickle.load(file)
            block.left_rotation_matrix = rotation_dict['left_rotation_matrix']
            block.right_rotation_matrix = rotation_dict['right_rotation_matrix']
            block.left_rotation_angle = rotation_dict['left_rotation_angle']
            block.right_rotation_angle = rotation_dict['right_rotation_angle']
    except FileNotFoundError:
        print('No rotation matrix file, create it')
    
    
def create_saccade_events_df(eye_data_df, speed_threshold, bokeh_verify_threshold=False, magnitude_calib=1, speed_profile=True):    
    """
    Detects saccade events in eye tracking data and computes relevant metrics.

    Parameters:
    - eye_data_df (pd.DataFrame): Input DataFrame containing eye tracking data.
    - speed_threshold (float): Threshold for saccade detection based on speed.

    Returns:
    - df (pd.DataFrame): Modified input DataFrame with added columns for speed and saccade detection.
    - saccade_events_df (pd.DataFrame): DataFrame containing information about detected saccade events.

    Steps:
    1. Calculate speed components ('speed_x', 'speed_y') based on differences in 'center_x' and 'center_y'.
    2. Compute the magnitude of the velocity vector ('speed_r').
    3. Create a binary column ('is_saccade') indicating saccade events based on the speed threshold.
    4. Determine saccade onset and offset indices and timestamps.
    5. Create a DataFrame ('saccade_events_df') with columns:
        - 'saccade_start_ind': Indices of saccade onset.
        - 'saccade_start_timestamp': Timestamps corresponding to saccade onset.
        - 'saccade_end_ind': Indices of saccade offset.
        - 'saccade_end_timestamp': Timestamps corresponding to saccade offset.
        - 'length': Duration of each saccade event.
    6. Calculate distance traveled and angles for each saccade event.
    7. Append additional columns to 'saccade_events_df':
        - 'magnitude': Magnitude of the distance traveled during each saccade.
        - 'angle': Angle of the saccade vector in degrees.
        - 'initial_x', 'initial_y': Initial coordinates of the saccade.
        - 'end_x', 'end_y': End coordinates of the saccade.

    Note: The original 'eye_data_df' is not modified; modified data is returned as 'df'.
    """
    df = eye_data_df
    df['speed_x'] = df['center_x'].diff()  # Difference between consecutive 'center_x' values
    df['speed_y'] = df['center_y'].diff()  # Difference between consecutive 'center_y' values
    
    # Step 2: Calculate magnitude of the velocity vector (R vector speed)
    df['speed_r'] = (df['speed_x']**2 + df['speed_y']**2)**0.5
    
    # Create a column for saccade detection
    df['is_saccade'] = df['speed_r'] > speed_threshold
    
    # create a saccade_on_off indicator where 1 is rising edge and -1 is falling edge by subtracting a shifted binary mask
    saccade_on_off = df.is_saccade.astype(int) - df.is_saccade.shift(periods=1,fill_value=False).astype(int)
    saccade_on_inds = np.where(saccade_on_off == 1)[0] - 1 # notice the manual shift here, chosen to include the first (sometimes slower) eye frame, just before saccade threshold crossing
    saccade_on_ms = df['ms_axis'].iloc[saccade_on_inds]
    saccade_on_timestamps = df['OE_timestamp'].iloc[saccade_on_inds]
    saccade_off_inds = np.where(saccade_on_off == -1)[0]
    saccade_off_timestamps = df['OE_timestamp'].iloc[saccade_off_inds]
    saccade_off_ms = df['ms_axis'].iloc[saccade_off_inds]
    
    saccade_dict = {'saccade_start_ind' :  saccade_on_inds ,
                    'saccade_start_timestamp': saccade_on_timestamps.values,
                    'saccade_end_ind':      saccade_off_inds,
                    'saccade_end_timestamp':saccade_off_timestamps.values,
                    'saccade_on_ms': saccade_on_ms.values,
                    'saccade_off_ms': saccade_off_ms.values}
    
    saccade_events_df = pd.DataFrame.from_dict(saccade_dict)
    saccade_events_df['length'] = saccade_events_df['saccade_end_ind'] - saccade_events_df['saccade_start_ind']
    # Drop columns used for intermediate steps
    df = df.drop(['is_saccade'], axis=1)
    
    distances = []
    angles = []
    speed_list = []
    diameter_list = []
    for index, row in tqdm.tqdm(saccade_events_df.iterrows()):
        saccade_samples = df.loc[(df['OE_timestamp'] >= row['saccade_start_timestamp']) & 
                                 (df['OE_timestamp'] <= row['saccade_end_timestamp'])]
        distance_traveled = saccade_samples['speed_r'].sum()
        if speed_profile:
            saccade_speed_profile = saccade_samples['speed_r'].values
            speed_list.append(saccade_speed_profile)
        saccade_diameter_profile = saccade_samples['pupil_diameter'].values
        diameter_list.append(saccade_diameter_profile)
        # Calculate angle from initial position to endpoint
        initial_position = saccade_samples.iloc[0][['center_x', 'center_y']]
        endpoint = saccade_samples.iloc[-1][['center_x', 'center_y']]
        overall_angle = np.arctan2(endpoint['center_y'] - initial_position['center_y'],
                           endpoint['center_x'] - initial_position['center_x'])
        
        angles.append(overall_angle)  
        distances.append(distance_traveled)
        
        
    
    saccade_events_df['magnitude_raw'] = np.array(distances)
    saccade_events_df['magnitude'] = np.array(distances) * magnitude_calib
    saccade_events_df['angle'] = np.where(np.isnan(angles), angles, np.rad2deg(angles) % 360) # Convert radians to degrees and ensure result is in [0, 360)
    start_ts = saccade_events_df['saccade_start_timestamp'].values
    end_ts = saccade_events_df['saccade_end_timestamp'].values
    saccade_start_df = df[df['OE_timestamp'].isin(start_ts)]
    saccade_end_df = df[df['OE_timestamp'].isin(end_ts)]
    start_x_coord = saccade_start_df['center_x']
    start_y_coord = saccade_start_df['center_y']
    end_x_coord = saccade_end_df['center_x']
    end_y_coord = saccade_end_df['center_y']
    saccade_events_df['initial_x'] = start_x_coord.values
    saccade_events_df['initial_y'] = start_y_coord.values
    saccade_events_df['end_x'] = end_x_coord.values
    saccade_events_df['end_y'] = end_y_coord.values
    saccade_events_df['calib_dx'] = (saccade_events_df['end_x'].values - saccade_events_df['initial_x'].values) * magnitude_calib
    saccade_events_df['calib_dy'] = (saccade_events_df['end_y'].values - saccade_events_df['initial_y'].values) * magnitude_calib
    if speed_profile:
        saccade_events_df['speed_profile'] = speed_list
    saccade_events_df['diameter_profile'] = diameter_list
    if bokeh_verify_threshold:
        bokeh_plotter(data_list=[df.speed_r], label_list=['Pupil Velocity'], peaks=saccade_on_inds)
        
    return df, saccade_events_df

# create a multi-animal block_collection:

def create_block_collections(animals, block_lists, experiment_path, bad_blocks=None):
    """
    Create block collections and a block dictionary from multiple animals and their respective block lists.

    Parameters:
    - animals: list of str, names of the animals.
    - block_lists: list of lists of int, block numbers corresponding to each animal.
    - experiment_path: pathlib.Path, path to the experiment directory.
    - bad_blocks: list of int, blocks to exclude. Default is an empty list.

    Returns:
    - block_collection: list of BlockSync objects for all specified blocks.
    - block_dict: dictionary where keys are block numbers as strings and values are BlockSync objects.
    """
    import UtilityFunctions_newOE as uf

    if bad_blocks is None:
        bad_blocks = []

    block_collection = []
    block_dict = {}

    for animal, blocks in zip(animals, block_lists):
        # Generate blocks for the current animal
        current_blocks = uf.block_generator(
            block_numbers=blocks,
            experiment_path=experiment_path,
            animal=animal,
            bad_blocks=bad_blocks
        )
        # Add to collection and dictionary
        block_collection.extend(current_blocks)
        for b in current_blocks:
            block_dict[f"{animal}_block_{b.block_num}"] = b
    
    return block_collection, block_dict


In [3]:


animals = ['PV_62','PV_126','PV_57']
block_lists = [[23,24,26,38],[7,8,9,10,11,12],[7,8,9,11,12,13]]
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
bad_blocks = [0]  # Example of bad blocks

block_collection, block_dict = create_block_collections(
    animals=animals, 
    block_lists=block_lists, 
    experiment_path=experiment_path, 
    bad_blocks=bad_blocks
)

instantiated block number 023 at Path: Z:\Nimrod\experiments\PV_62\2023_04_27\block_023, new OE version
Found the sample rate for block 023 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 023
got it!
instantiated block number 024 at Path: Z:\Nimrod\experiments\PV_62\2023_04_27\block_024, new OE version
Found the sample rate for block 024 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 024
got it!
instantiated block number 026 at Path: Z:\Nimrod\experiments\PV_62\2023_04_27\block_026, new OE version
Found the sample rate for block 026 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 026
got it!
instantiated block number 038 at Path: Z:\Nimrod\experi

In [4]:
for block in block_collection:
    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)

    # if the code fails here, go to manual synchronization
    block.import_manual_sync_df()
    block.read_dlc_data()
    block.calibrate_pixel_size(10)
    #load_eye_data_2d_w_rotation_matrix(block) #should be integrated again... later
    block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data_degrees_raw.csv', index_col=0, engine='python')
    block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_degrees_raw.csv', index_col=0, engine='python')
    
    # calibrate pupil diameter:
    if 'pupil_diameter' not in block.left_eye_data.columns:
        block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax 
        block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax
        block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
        block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size

running parse_open_ephys_events...
block 023 has a parsed events file, reading...
Getting eye brightness values for block 023...
Found an existing file!
Eye brightness vectors generation complete.
blocksync_df loaded from analysis folder
eye_brightness_df loaded from analysis folder
eye dataframes loaded from analysis folder
got the calibration values from the analysis folder
running parse_open_ephys_events...
block 024 has a parsed events file, reading...
Getting eye brightness values for block 024...
Found an existing file!
Eye brightness vectors generation complete.
blocksync_df loaded from analysis folder
eye_brightness_df loaded from analysis folder
eye dataframes loaded from analysis folder
got the calibration values from the analysis folder
running parse_open_ephys_events...
block 026 has a parsed events file, reading...
Getting eye brightness values for block 026...
Found an existing file!
Eye brightness vectors generation complete.
blocksync_df loaded from analysis folder
eye_

In [6]:
block.left_eye_data.columns

Index(['Unnamed: 0.1', 'OE_timestamp', 'eye_frame', 'ms_axis', 'center_x',
       'center_y', 'phi', 'width', 'height', 'major_ax', 'minor_ax', 'ratio',
       'pupil_diameter_pixels', 'pupil_diameter', 'ratio2', 'phi_ellipse',
       'k_phi', 'k_theta'],
      dtype='object')

In [10]:
# over here, I am creating the new rotation function for the eye_data dfs - this should be integrated into the blocksync class later: 
import cv2
import numpy as np
import pandas as pd

def rotate_single_eye_data(block, eye, new_rotation_angle):
    """
    Rotate the eye data for a single eye by counter-rotating the current rotation and applying an additional rotation.
    
    This function:
      1. Determines the center of the frame by reading the first frame of the corresponding eye video.
      2. Computes the net rotation as: net_rotation = new_rotation_angle - current_rotation_angle.
      3. Applies the net rotation to the 'center_x' and 'center_y' coordinates of the eye data.
      4. Updates the 'phi' (ellipse orientation) by subtracting the net rotation.
      5. Updates the BlockSync object with the newly rotated data and new rotation angle.
    
    Parameters:
        block : BlockSync object
            The BlockSync instance containing the eye data and rotation parameters.
        eye : str
            Either "left" or "right", indicating which eye's data to rotate.
        new_rotation_angle : float
            The desired new rotation angle (in degrees) after applying the transformation.
    
    Returns:
        None. The function updates the block.left_eye_data or block.right_eye_data and the corresponding rotation angle in-place.
    """
    # Select the appropriate data and video list based on the eye.
    if eye.lower() == 'left':
        eye_data = block.left_eye_data
        current_angle = block.left_rotation_angle
        video_list = block.le_videos
    elif eye.lower() == 'right':
        eye_data = block.right_eye_data
        current_angle = block.right_rotation_angle
        video_list = block.re_videos
    else:
        raise ValueError("Eye must be 'left' or 'right'")
    
    # Ensure that there is at least one video available.
    if not video_list:
        raise ValueError(f"No video available for {eye} eye.")
    
    # Open the first video to determine frame dimensions and compute the center.
    video_path = video_list[0]
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {video_path}")
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    center = (frame_width / 2, frame_height / 2)
    cap.release()
    
    # Compute the net rotation angle: first counter-rotate (undo current) then apply new rotation.
    net_rotation = new_rotation_angle - current_angle
    
    # Compute the affine transformation matrix for the net rotation.
    M = cv2.getRotationMatrix2D(center, net_rotation, 1.0)
    
    # Extract (center_x, center_y) from the DataFrame and convert to float32.
    coords = eye_data[['center_x', 'center_y']].values.astype(np.float32)
    # Convert coordinates to homogeneous form by adding a column of ones.
    ones = np.ones((coords.shape[0], 1), dtype=np.float32)
    coords_homogeneous = np.hstack([coords, ones])
    # Apply the rotation matrix.
    rotated_coords = np.dot(M, coords_homogeneous.T).T
    # Update the DataFrame with the new coordinates.
    eye_data['center_x'] = rotated_coords[:, 0]
    eye_data['center_y'] = rotated_coords[:, 1]
    
    # Update the ellipse orientation (phi) by subtracting the net rotation.
    eye_data['phi'] = (eye_data['phi'] - net_rotation) % 360
    
    # Update the BlockSync object with the rotated data and new rotation angle.
    if eye.lower() == 'left':
        block.left_eye_data = eye_data
        block.left_rotation_angle = new_rotation_angle
    else:
        block.right_eye_data = eye_data
        block.right_rotation_angle = new_rotation_angle


In [11]:
rotate_single_eye_data(block,'left',new_rotation_angle=0)

In [12]:
import cv2
import numpy as np

def verify_frame_alignment_scroll(block, eye, path_to_video=False, xflip=False, phi_in_radians=False):
    """
    Allows interactive scrolling through video frames via a trackbar to verify that the rotated 
    video frames and the corresponding rotated eye data (ellipse overlay) are correctly aligned.
    
    The function:
      - Opens the video for the specified eye (using block.le_videos or block.re_videos).
      - Creates an OpenCV window with a "Frame" trackbar to select the frame index.
      - For each selected frame, it reads the raw frame, applies the stored rotation (using the 
        stored rotation angle for that eye to compute a transformation matrix based on the frame's center),
        - Retrieves the corresponding ellipse data (center_x, center_y, phi, width, height) for that frame,
        - Draws the ellipse on the rotated frame, and displays the result.
    
    Parameters:
        block : BlockSync object
            Contains the video lists and rotated eye data DataFrames (left_eye_data/right_eye_data),
            as well as the stored rotation angles (left_rotation_angle/right_rotation_angle).
        eye : str
            Either "left" or "right", specifying which eye’s video and data to use.
        path_to_video : str or False, optional
            If provided, overrides the video path stored in the block.
        xflip : bool, optional
            If True, the frame is flipped horizontally.
        phi_in_radians : bool, optional
            If True, converts the ellipse orientation to radians before drawing.
    
    Returns:
        None. The function only displays the video; press 'q' to exit.
    """
    # Select the appropriate video and ellipse data based on the eye.
    if eye.lower() == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.left_eye_data
        stored_angle = block.left_rotation_angle
    elif eye.lower() == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.right_eye_data
        stored_angle = block.right_rotation_angle
    else:
        raise ValueError("eye must be 'left' or 'right'")
    
    if path_to_video:
        video_path = path_to_video

    # Open the video.
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Get video frame count and dimensions.
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    ret, temp_frame = cap.read()
    if not ret:
        print("Error reading first frame.")
        cap.release()
        return
    frame_height, frame_width = temp_frame.shape[:2]
    center = (frame_width / 2, frame_height / 2)
    
    # Create window and trackbar.
    window_name = "Frame Alignment Verification"
    cv2.namedWindow(window_name)
    cv2.createTrackbar("Frame", window_name, 0, frame_count - 1, lambda x: None)

    while True:
        # Read current frame index from trackbar.
        frame_idx = cv2.getTrackbarPos("Frame", window_name)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            continue

        if xflip:
            frame = cv2.flip(frame, 1)

        # Compute the transformation matrix using the stored rotation angle.
        # (This rotates the raw video frame so that its orientation matches the rotated eye data.)
        M = cv2.getRotationMatrix2D(center, stored_angle, 1.0)
        rotated_frame = cv2.warpAffine(frame, M, (frame_width, frame_height))
        
        # Retrieve the ellipse data for the current frame.
        current_frame_num = frame_idx
        try:
            # Assumes the eye_data DataFrame has an "eye_frame" column matching the frame number.
            idx = ellipse_dataframe.query("eye_frame == @current_frame_num").index[0]
            ellipse_data = ellipse_dataframe.loc[idx]
            
            # Use the rotated coordinates as stored in the DataFrame.
            ell_center = (int(ellipse_data['center_x']), int(ellipse_data['center_y']))
            ell_width = int(ellipse_data['width'])
            ell_height = int(ellipse_data['height'])
            ell_phi = float(ellipse_data['phi'])
            if phi_in_radians:
                ell_phi = np.deg2rad(ell_phi)
            
            # Draw the ellipse on the rotated frame.
            cv2.ellipse(rotated_frame, ell_center, (ell_width, ell_height), ell_phi, 0, 360, (0, 255, 0), 2)
        except IndexError:
            # If no ellipse data exists for this frame, simply continue.
            pass

        # Display the frame along with an overlay of the current frame index.
        cv2.putText(rotated_frame, f'Frame: {frame_idx}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow(window_name, rotated_frame)

        # Wait a short moment and break if 'q' is pressed.
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
verify_frame_alignment_scroll(block, 'left', path_to_video=False, xflip=True, phi_in_radians=False)

In [10]:
cv2.destroyAllWindows()

In [4]:
# This function zeros the previous iteration of rotation-correction - we do this to create a new paradigm where we load data raw, and rotate it subsequently according to pre-defined angles for each eye. 
# first step is to check this works

import cv2
import numpy as np
import pandas as pd
import pickle

def restore_original_eye_data(block, save_to_disk=True):
    """
    Restores the original orientation of the rotated eye data CSV files by counter-rotating 
    the data using the stored rotation parameters. The function:
    
      1. Loads the left and right eye data CSV files (if not already loaded).
      2. Loads the rotation parameters (left/right rotation angles and matrices) from the 
         'rotate_eye_data_params.pkl' file.
      3. For each eye, determines the frame center using the first frame of the corresponding video.
      4. Computes the inverse transformation matrix using the negative stored rotation angle.
      5. Applies this transformation to the 'center_x' and 'center_y' columns.
      6. Updates the ellipse orientation ('phi') by adding the stored rotation angle.
      7. Updates the BlockSync object in memory and (optionally) saves the restored data to new CSV files.
    
    Parameters:
        block : BlockSync object
            The BlockSync object containing:
              - analysis_path (a pathlib.Path)
              - le_videos and re_videos (lists of video paths)
              - left_eye_data and right_eye_data attributes (if not, they will be loaded)
        save_to_disk : bool, optional
            If True, the restored data is saved as 'left_eye_data_original.csv' and 
            'right_eye_data_original.csv' in the analysis folder.
    
    Returns:
        None.
    """
    
    # Step 1: Load eye data CSV files if not already loaded.
    try:
        if block.left_eye_data is None:
            block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data.csv', index_col=0, engine='python')
        if block.right_eye_data is None:
            block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data.csv', index_col=0, engine='python')
    except Exception as e:
        print("Error loading eye data CSV files:", e)
        return
    
    # Step 2: Load rotation parameters.
    try:
        with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'rb') as f:
            rotation_dict = pickle.load(f)
            block.left_rotation_angle = rotation_dict.get('left_rotation_angle', 0)
            block.right_rotation_angle = rotation_dict.get('right_rotation_angle', 0)
            block.left_rotation_matrix = rotation_dict.get('left_rotation_matrix', None)
            block.right_rotation_matrix = rotation_dict.get('right_rotation_matrix', None)
    except FileNotFoundError:
        print("Rotation parameters file not found. Cannot restore original orientation.")
        return
    
    # Helper function to restore eye data.
    def restore_eye_data(eye_data, video_path, stored_angle):
        # Open the video to determine frame dimensions.
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {video_path}")
        ret, frame = cap.read()
        if not ret:
            cap.release()
            raise ValueError(f"Cannot read first frame from video: {video_path}")
        frame_height, frame_width = frame.shape[:2]
        center = (frame_width / 2, frame_height / 2)
        cap.release()
        
        # Compute the inverse rotation:
        # To restore the original orientation, apply a rotation of -stored_angle.
        net_rotation = -stored_angle
        
        # Compute the inverse transformation matrix.
        M_inv = cv2.getRotationMatrix2D(center, net_rotation, 1.0)
        
        # Apply the transformation to the (center_x, center_y) coordinates.
        coords = eye_data[['center_x', 'center_y']].values.astype(np.float32)
        ones = np.ones((coords.shape[0], 1), dtype=np.float32)
        coords_hom = np.hstack([coords, ones])
        restored_coords = np.dot(M_inv, coords_hom.T).T
        
        # Update the DataFrame with restored coordinates.
        eye_data['center_x'] = restored_coords[:, 0]
        eye_data['center_y'] = restored_coords[:, 1]
        
        # Update the ellipse orientation (phi) by adding the stored angle.
        # (If the data were rotated by 'stored_angle', restoring means: original_phi = rotated_phi + stored_angle.)
        eye_data['phi'] = (eye_data['phi'] + stored_angle) % 360
        
        return eye_data

    # Step 3: Restore left and right eye data.
    try:
        left_video_path = block.le_videos[0]
        right_video_path = block.re_videos[0]
    except Exception as e:
        print("Error accessing video paths from block:", e)
        return
    
    restored_left = restore_eye_data(block.left_eye_data, left_video_path, block.left_rotation_angle)
    restored_right = restore_eye_data(block.right_eye_data, right_video_path, block.right_rotation_angle)
    
    block.left_eye_data = restored_left
    block.right_eye_data = restored_right

    # Step 4: Optionally, save the restored data to new CSV files.
    if save_to_disk:
        left_save_path = block.analysis_path / 'left_eye_data_original.csv'
        right_save_path = block.analysis_path / 'right_eye_data_original.csv'
        restored_left.to_csv(left_save_path)
        restored_right.to_csv(right_save_path)
        print(f"Restored left eye data saved to {left_save_path}")
        print(f"Restored right eye data saved to {right_save_path}")


In [6]:
for block in block_collection:
    restore_original_eye_data(block,save_to_disk=True)

Restored left eye data saved to Z:\Nimrod\experiments\PV_62\2023_04_27\block_023\analysis\left_eye_data_original.csv
Restored right eye data saved to Z:\Nimrod\experiments\PV_62\2023_04_27\block_023\analysis\right_eye_data_original.csv
Restored left eye data saved to Z:\Nimrod\experiments\PV_62\2023_04_27\block_024\analysis\left_eye_data_original.csv
Restored right eye data saved to Z:\Nimrod\experiments\PV_62\2023_04_27\block_024\analysis\right_eye_data_original.csv
Restored left eye data saved to Z:\Nimrod\experiments\PV_62\2023_04_27\block_026\analysis\left_eye_data_original.csv
Restored right eye data saved to Z:\Nimrod\experiments\PV_62\2023_04_27\block_026\analysis\right_eye_data_original.csv
Restored left eye data saved to Z:\Nimrod\experiments\PV_62\2023_05_01\block_038\analysis\left_eye_data_original.csv
Restored right eye data saved to Z:\Nimrod\experiments\PV_62\2023_05_01\block_038\analysis\right_eye_data_original.csv
Restored left eye data saved to Z:\Nimrod\experiments\PV

In [8]:
import pandas as pd
import numpy as np

def load_original_eye_data_and_reset_rotation(block):
    """
    Loads the original eye data CSV files ('left_eye_data_original.csv' and 'right_eye_data_original.csv')
    and assigns them to block.left_eye_data and block.right_eye_data. Then resets the rotation parameters
    by setting block.left_rotation_angle and block.right_rotation_angle to 0 and assigning identity
    matrices to block.left_rotation_matrix and block.right_rotation_matrix.
    
    This does not modify the original files; it only updates the attributes of the BlockSync object.
    
    Parameters:
        block : BlockSync object
            The BlockSync instance, which must have an 'analysis_path' attribute.
    
    Returns:
        None. The function updates block.left_eye_data, block.right_eye_data, 
              block.left_rotation_angle, block.right_rotation_angle,
              block.left_rotation_matrix, and block.right_rotation_matrix.
    """
    try:
        left_path = block.analysis_path / 'left_eye_data_original.csv'
        right_path = block.analysis_path / 'right_eye_data_original.csv'
        
        block.left_eye_data = pd.read_csv(left_path, index_col=0, engine='python')
        block.right_eye_data = pd.read_csv(right_path, index_col=0, engine='python')
        
        print(f"Successfully loaded original eye data from:\n  {left_path}\n  {right_path}")
    except Exception as e:
        print("Error loading original eye data:", e)
        return
    
    # Reset the stored rotation angles to zero.
    block.left_rotation_angle = 0
    block.right_rotation_angle = 0
    
    # Reset the rotation matrices to identity.
    # For an affine transformation (used by cv2.warpAffine), the identity matrix is:
    # [[1, 0, 0],
    #  [0, 1, 0]]
    identity_matrix = np.array([[1, 0, 0],
                                [0, 1, 0]], dtype=float)
    
    block.left_rotation_matrix = identity_matrix.copy()
    block.right_rotation_matrix = identity_matrix.copy()
    
    print("Rotation parameters reset: angles set to 0 and rotation matrices set to identity.")
load_original_eye_data_and_reset_rotation(block)

Successfully loaded original eye data from:
  Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis\left_eye_data_original.csv
  Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis\right_eye_data_original.csv
Rotation parameters reset: angles set to 0 and rotation matrices set to identity.


In [1]:
def append_angle_data(eye_df, new_df):
    """
    Appends the kinematics columns (phi and theta) from new_df to eye_df.
    The function renames 'phi' to 'k_phi' and 'theta' to 'k_theta', then merges
    on the shared 'OE_timestamp' column.
    
    Parameters:
    - eye_df: pandas DataFrame containing the eye tracking data.
    - new_df: pandas DataFrame containing the new kinematics data with columns
              'phi' and 'theta' along with 'OE_timestamp' (and possibly others).
    
    Returns:
    - merged_df: pandas DataFrame resulting from merging the new kinematics data
                 into eye_df.
    """
    # Select the necessary columns and rename them
    angle_data = new_df[['OE_timestamp', 'phi', 'theta']].rename(
        columns={'phi': 'k_phi', 'theta': 'k_theta'}
    )
    
    # Merge on OE_timestamp using a left join to preserve all rows in eye_df
    merged_df = pd.merge(eye_df, angle_data, on='OE_timestamp', how='left')
    
    return merged_df

for block in block_collection:
    print(block)
    try:
        left_angles = pd.read_csv([i for i in block.analysis_path.iterdir() if ('left_kerr_angles_current.csv' in str(i)) ][0])
        right_angles = pd.read_csv([i for i in block.analysis_path.iterdir() if ('right_kerr_angles_current.csv' in str(i)) ][0])
    except IndexError:
        print(f'{block} has a problem')
    
    block.left_eye_data = append_angle_data(block.left_eye_data,left_angles)
    block.right_eye_data = append_angle_data(block.right_eye_data,right_angles)

for block in block_collection:
    block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data_original.csv')
    block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data_original.csv')
    

NameError: name 'block_collection' is not defined